# CSVS to Hepfile
Imagine we have a database like structure with multiple csvs all connected by a common ID. See the household exmaple here: https://hepfile.readthedocs.io/en/latest/fundamentals.html#a-toy-example

In [1]:
import os, glob
import numpy as np
import pandas as pd
import awkward as ak
import hepfile as hf

In [2]:

datapath = os.path.join('/', 'home', 'nfranz', 'research', 'hepfile', 'docs', 'example_nb', '*.csv')
files = glob.glob(datapath)
common_key = 'Household ID'
group_names = ['Residences', 'People', 'Vehicles']
awk = hf.csv_tools.csv_to_hepfile(files, common_key, outfile='test-csv.h5', group_names=group_names)

----------------------------------------------------
Slashes / are not allowed in dataset names
Replacing / with - in dataset name Highest degree/grade
The new name will be Highest degree-grade
----------------------------------------------------
----------------------------------------------------
Slashes / are not allowed in dataset names
Replacing / with - in dataset name Gas/electric/human powered
The new name will be Gas-electric-human powered
----------------------------------------------------
----------------------------------------------------
Slashes / are not allowed in dataset names
Replacing / with - in dataset name House/apartment/condo
The new name will be House-apartment-condo
----------------------------------------------------
Writing the hdf5 file from the awkward array...
Metadata added


/home/nfranz/research/hepfile/src/hepfile/write.py:658: UserWarning: Residences/nResidences and _SINGLETONS_GROUP_/COUNTER have differing numbers of entries!
  warnings.warn(f"{countername} and {prevcounter} have differing numbers of entries!")
/home/nfranz/research/hepfile/src/hepfile/write.py:658: UserWarning: People/nPeople and Residences/nResidences have differing numbers of entries!
  warnings.warn(f"{countername} and {prevcounter} have differing numbers of entries!")


In [3]:
data, _ = hf.load('test-csv.h5', return_awkward=True)

data

Building the indices...

Built the indices!
Data is read in and input file is closed.


ValueError: dict of arrays in ak.Array constructor must have arrays of equal length (3 vs 4)